In [1]:
import os
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from models.resnet import resnet50
from my_dataset import MyDataset
from my_lossfunc import JointLoss, MultilabelLoss, DiscriminativeLoss
from my_transform import data_transforms
from scipy.spatial.distance import pdist
from torch.utils.data import DataLoader
from tqdm import tqdm
from utils import *

os.environ["CUDA_VISIBLE_DEVICES"] = "1,2,3"
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print('Device:', DEVICE)


BASE = '/home/zengrui/datasets'
DUKE_DIR_TRAIN = f'{BASE}/ReID_Duke/bounding_box_train'
DUKE_DIR_TEST = f'{BASE}/ReID_Duke/bounding_box_test'
DUKE_IMG_AMOUNT = 16522
DUKE_ID_AMOUNT = 702
MARKET_DIR_TRAIN = f'{BASE}/ReID_Market/bounding_box_train'
MARKET_DIR_TEST = f'{BASE}/ReID_Market/bounding_box_test'
MARKET_IMG_AMOUNT = 12936
MARKET_ID_AMOUNT = 751
ML_PATH = 'data/ml_Market.dat'
PRETRAIN_PATH = 'data/pretrained_weight.pkl'
PRETRAIN_OUT_PATH = 'data/pretrained_weight_{}.pkl'

BATCH_SIZE = 96
EPOCH = 30
LR = 0.1

BETA = 0.2
LAMB1 = 2e-4
LAMB2 = 50
MARGIN = 1
SCALA_CE = 30

Device: cuda


## Prepare Work

In [2]:
# data loader
data_loader = {
    'source': DataLoader(
        dataset=MyDataset(DUKE_DIR_TRAIN, 
                          transform=data_transforms('train')),
        batch_size=BATCH_SIZE,
        shuffle=True,
    ),
    'target': DataLoader(
        dataset=MyDataset(MARKET_DIR_TRAIN,
                          transform=data_transforms('train'),
                          require_view=True),
        batch_size=BATCH_SIZE,
        shuffle=True,
    ),
}
print('data_loader: ok.')

data_loader: ok.


## Train

In [3]:
class Trainer(object):
    def __init__(self):
        
        # 网络
        self.net = resnet50(pretrained=False, 
                            num_classes=DUKE_ID_AMOUNT)
        self.net = nn.DataParallel(self.net).to(DEVICE)
        if PRETRAIN_PATH is not None and os.path.exists(PRETRAIN_PATH):
            self.net.load_state_dict(torch.load(PRETRAIN_PATH))
            print('Pre-trained model loaded.')
        else:
            print('Pre-trained model not found. Train from scratch.')
            
        # 损失
        self.mdl_loss = DiscriminativeLoss(0.001).to(DEVICE)
        self.al_loss = nn.CrossEntropyLoss().to(DEVICE)
        self.rj_loss = JointLoss(MARGIN).to(DEVICE)  # lack 1 param
        self.cml_loss = MultilabelLoss(BATCH_SIZE).to(DEVICE)
        
        # 优化器
        self.optimizer = torch.optim.SGD(
            self.net.parameters(), lr=LR, momentum=0.9)
        self.lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(
            self.optimizer, milestones=[int(EPOCH / 8 * 5), int(EPOCH / 8 * 7)])
        
        # 存储器
        self.ml_mem = torch.zeros(MARKET_IMG_AMOUNT, DUKE_ID_AMOUNT)
        self.inited = self.ml_mem.sum(dim=1) != 0
    
    def train(self):
        for epoch in range(EPOCH):
            self.train_epoch(epoch)
    
    def train_epoch(self, epoch):
        count = 0
        running_loss = {'total': 0,
                        'src': 0,
                        'st': 0,
                        'ml': 0,
                        'tgt': 0,
                       }
        if not self.mdl_loss.initialized:  # ...
            self.init_losses(data_loader['target'])
        
        with tqdm(total=len(data_loader['source'])) as pbar:
            tgt_iter = iter(data_loader['target'])
            for step, (ax, ay) in enumerate(data_loader['source']):
                # a - source, b - target
                ax = ax.to(DEVICE)
                ay = ay.to(DEVICE)
                try:
                    b = next(tgt_iter)
                except StopIteration:
                    tgt_iter = iter(data_loader['target'])
                    b = next(tgt_iter)
                (bx, by, b_view, b_idx) = b
                bx, by, b_view = bx.to(DEVICE), by.to(DEVICE), b_view.to(DEVICE)

                a_f, a_sim, _ = self.net(ax)
                b_f, b_sim, _ = self.net(bx)

                loss_src = self.al_loss(a_sim * SCALA_CE, ay)  # 有监督 交叉熵
                
                agents = self.net.module.fc.weight.renorm(2, 0, 1e-5).mul(1e5)  # 归一化 shape=(702, 2048)
                
                loss_st = self.rj_loss(agents.detach(), a_f, a_sim.detach(), ay, 
                                       b_f, b_sim.detach())
                
                with torch.no_grad():
                    ml = F.softmax(b_f.mm(agents.t_() * SCALA_CE), dim=1)  # t_(): 转置并inplace
#                 loss_ml = self.cml_loss(torch.log(ml), b_view)

                loss_ml = torch.Tensor([0]).to(DEVICE)
    
                if epoch < 1:  # 为什么第一轮不算 mdl_loss 呢
                    loss_tgt = torch.Tensor([0]).to(DEVICE)
                else:
                    ml_cpu = ml.detach().cpu()
                    is_inited_batch = self.inited[b_idx]
                    inited_idx = b_idx[is_inited_batch]
                    uninited_idx = b_idx[~is_inited_batch]
                    self.ml_mem[uninited_idx] = ml_cpu[~is_inited_batch]  # 0标签满更新
                    self.inited[uninited_idx] = True
                    self.ml_mem[inited_idx] = 0.9 * self.ml_mem[inited_idx] \
                                            + 0.1 * ml_cpu[is_inited_batch]  # 非空标签小更新
                    loss_tgt = self.mdl_loss(b_f, self.ml_mem[b_idx], by)

                self.optimizer.zero_grad()
                loss_total = loss_tgt + LAMB1 * loss_ml + LAMB2 * (loss_src + BETA * loss_st)
                loss_total.backward()
                self.optimizer.step()

                count += 1
                loss_cpu = float(loss_total.data.cpu().numpy())
                loss_src_cpu = float(loss_src.data.cpu().numpy())
                loss_st_cpu = float(loss_st.data.cpu().numpy())
                loss_ml_cpu = float(loss_ml.data.cpu().numpy())
                loss_tgt_cpu = float(loss_tgt.data.cpu().numpy())
                running_loss['total'] += loss_cpu
                running_loss['src'] += loss_src_cpu
                running_loss['st'] += loss_st_cpu
                running_loss['ml'] += loss_ml_cpu
                running_loss['tgt'] += loss_tgt_cpu
                pbar.set_description('Loss: %.4f (%.4f + %.4f + %.4f + %.4f)' 
                                     % (loss_cpu, loss_src_cpu, loss_st_cpu, loss_ml_cpu, loss_tgt_cpu))
                pbar.update()

            self.lr_scheduler.step()
            for k in running_loss.keys():
                running_loss[k] /= count
            print('Epoch: %d, Loss: %.4f (%.4f + %.4f + %.4f + %.4f)' 
                  % (epoch, running_loss['total'], running_loss['src'], 
                     running_loss['st'], running_loss['ml'], running_loss['tgt']))
            
    def init_losses(self, tgt_loader):
        print('Initializing losses ....')
        if os.path.isfile(ML_PATH):
            (ml, view, pairwise_agreements) = torch.load(ML_PATH)
            print('ml loaded.')
        else:
            print('ml not found, computing ....')
            sim, _, views = extract_features(
                data_loader['target'], self.net, index_feature=1, return_numpy=False)
            ml = F.softmax(sim * SCALA_CE, dim=1)
            ml_np = ml.cpu().numpy()
            pairwise_agreements = 1 - pdist(ml_np, 'minkowski', p=1) / 2  # 相似比较特征
            print('ml saving to %s...' % ML_PATH)
            torch.save((ml, views, pairwise_agreements), ML_PATH)
        log_ml = torch.log(ml)
        # ...
        self.mdl_loss.init_threshold(pairwise_agreements)
        print('mdl_loss threshold inited.')
        
    def save_model(self, cover=False):
        if cover:
            torch.save(self.net.state_dict(), PRETRAIN_PATH)
        else:
            torch.save(self.net.state_dict(), PRETRAIN_OUT_PATH.format(time.time()))
        print('Model weight saved.')

In [4]:
trainer = Trainer()
trainer.train()

Pre-trained model not found. Train from scratch.
Initializing losses ....
ml loaded.


  0%|          | 0/173 [00:00<?, ?it/s]

mdl_loss threshold inited.


Loss: 393.0692 (7.5515 + 1.5497 + 0.0000 + 0.0000): 100%|██████████| 173/173 [06:00<00:00,  2.08s/it] 
  0%|          | 0/173 [00:00<?, ?it/s]

Epoch: 0, Loss: 572.2653 (11.2167 + 1.1430 + 0.0000 + 0.0000)


...

Epoch: 27, Loss: 69.8352 (1.1436 + 1.2032 + 0.0000 + 0.6222)


Loss: 120.7345 (2.1270 + 1.3754 + 0.0000 + 0.6305): 100%|██████████| 173/173 [06:50<00:00,  2.37s/it]
  0%|          | 0/173 [00:00<?, ?it/s]

Epoch: 28, Loss: 69.8719 (1.1446 + 1.2031 + 0.0000 + 0.6090)


Loss: 195.6564 (3.6103 + 1.4575 + 0.0000 + 0.5658): 100%|██████████| 173/173 [06:47<00:00,  2.36s/it]

Epoch: 29, Loss: 70.3622 (1.1542 + 1.2042 + 0.0000 + 0.6111)


In [ ]:
trainer.save_model()
trainer.save_model(cover=True)